In [1]:
import pickle
from tqdm import tqdm
import gc

entities_file_name = '../data/entities.87890.pickle'

In [2]:
def find_pickled_offsets(fileread):
    offsets = []
    new_row_offset = -1
    while True:
        # 80 = PROTOCOL (pickle)
        # 03 = VERSION 03
        # 636e756d7079 = cnumpy
        new_row_offset = fileread.find((0x8003636e756d7079).to_bytes(8, byteorder='big'), new_row_offset + 1)
        if new_row_offset == -1:
            break
        offsets.append(new_row_offset)
    return offsets

def parse_pickled_object(offset, infile, counters):
    
    if counters['total'] % 5000 == 0:
        gc.collect()
    
    try:
        counters['total'] += 1
        infile.seek(offset)
        row = pickle.load(infile)
        # check if entities are present
        if(row[1] == None):
            counters['error_count'] += 1
            return None
        else:
            return row
    except TypeError:
        counters['error_parser_count'] += 1
        return None

def read_entities_from_disk():
    counters = {'error_count': 0, 'error_parser_count': 0, 'total': 0}
    try:
        with open(entities_file_name, 'rb') as infile:
            fileread = infile.read()
            offsets = tqdm(find_pickled_offsets(fileread))
            del fileread
            return [parse_pickled_object(offset, infile, counters) for offset in offsets]
    except EOFError:
        pass
    return my_objects

class DummyStdErr:
    def flush(dummy):
        pass
    
# Fix Parsing error ¯\_(ツ)_/¯
from grpc._channel import _Rendezvous
_Rendezvous.__del__ = lambda x: None

In [3]:
import sys

entities = read_entities_from_disk()
print(len(entities))

 13%|█▎        | 12992/103235 [11:45<1:21:36, 18.43it/s]               | 6992/103235 [03:00<41:20, 38.79it/s]          

KeyboardInterrupt: 

In [ ]:
# Format
# (id, EntitesObject, None)
entities[34]